In [1]:
%run input/Format.ipynb
!rm output/delta_fits/?/?/*
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 0
test = 1
sim = 0
sep_charge = 0
Min_Entries = 100

In [3]:
isigma = 2
N_files = 1
N_centr = 5
N_hists = 4
N_layers = 4
N_ilyaers = 8
is_e = 0
iOption1 = is_sigmalised
xaxis=["phi","theta"]
centralities = ["0-20%","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,4,5]
layer_bins = [1,4,5,8,9,12,13,16]
if sep_charge: layer_bins, N_layers = [1,1,5,5,9,9,13,13,2,2,6,6,10,10,14,14,3,3,7,7,11,11,15,15,4,4,8,8,12,12,16,16], 16
if sep_charge: layer_bins, N_layers = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,11,11,12,12,13,13,14,14,15,15,16,16], 16
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/NewHadron/"
file_names="missalign_hadron_new8_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el","dthe_hist_el","sdphi_hist_el","sdthe_hist_el"]

In [4]:
if sim :
    file_path="../sim/output/Newembed/deltas/"
    file_names="piminus_DCA_50M_V1" #piminus_DCA_50M_V0_100mrad ccbar_50M_v0 piminus_DCA_50M_V0

In [5]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if not test: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    if test: infile = root.TFile.Open(file_path+file_names+".root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{centrality[2*icentr]-1}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/NewHadron/missalign_hadron_new8_rg0-80.root


In [6]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.8,1.0,1.4,2.0,2.8,4.9])
#pt_boarders = array( 'd', [0.5,0.6,0.8,1.0,1.4,2.0,2.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.7, 0.9, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0, 1.4, 2.0, 2.8, 4.9])


In [7]:
boards_tot = [[-0.1,0.1],[-0.1,0.1]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
if iOption1==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]

In [8]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=1
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):

        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)
        if emc_var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = 0.5* (pt_boarders[p]+pt_boarders[p+1])
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.

        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus{seed+p}', 'gaus',  boards_fit[int(seed0%2)][0], boards_fit[int(seed0%2)][1])
        g_pol = root.TF1( f'g_pol{seed+p}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(seed0%2)][0],boards_tot[int(seed0%2)][1])
        Format_Func(g_gaus,3,4)
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}', 'gaus(0)+[3]+[4]*x*(abs(x)/x-1)+[5]*x*(abs(x)/x+1)',boards_tot[int(seed0%2)][0], boards_tot[int(seed0%2)][1])
        Format_Func(total,2,6)


        if fits_mean!=0 and fits_sigma!=0:
            mean_ii=fits_sigma.Eval(pt[p])
            sigma_ii=fits_sigma.Eval(pt[p])
            g_gaus.SetParameter(1,sigma_ii)
            g_gaus.SetParameter(2,sigma_ii)
            g_gaus.SetParLimits(1,mean_ii*0.8-0.05,mean_ii*1.2+0.05)
            g_gaus.SetParLimits(2,sigma_ii*0.5,sigma_ii*1.5)
        else:
            g_gaus.SetParameter(1,0.)
            g_gaus.SetParameter(2,boards_fit[int(seed0%2)][1])

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(seed0%2)][0]/10,boards_tot[int(seed0%2)][1]/10)
        total.SetParLimits(2,boards_fit[int(seed0%2)][1]/10,boards_tot[int(seed0%2)][1]/2)

        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]

        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1))

        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        

        Sigma_local_err.append(total.GetParError(2)+Sigma_local[-1]/(2+abs(int_fg_local[-1]/4/total.GetParameter(2)))**0.5)

        m2_name=str(icentr)+"/"+str(int(ilayer/4))+"/emc_"+str(round(pt[p],1))+"_"+str(ilayer%4)

        c1.SaveAs( "output/delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [9]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in range(N_layers):
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+iOption1*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [10]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5,6,7], i, isigma) for i in range(Ntr)])
pool.close()

In [11]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
            pt_arr0.append(output_array[ifile][icentr][ilayer][6])
            pt_arr_err0.append(output_array[ifile][icentr][ilayer][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [12]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
colors=[1,2,4,6,root.kGreen+2]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)+x*[3]",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.004)
            fit_functions1[-1].SetParameter(1,0.022)
            fit_functions1[-1].SetParameter(2,-3)
            #fit_functions1[-1].SetParLimits(2,-4.126,-1.023)
            fit_functions1[-1].SetParameter(3,-0.001)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [13]:
g_sigma = []
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1500,900)
    c3.Divide(3,2)
    g_sigma_f = []
    min, max = 0., 0.03
    if iOption1==1: max, min = 4.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        g_sigma_centr = []
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "#sigma #"+xaxis[iOption0], 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            N_pt=len(pt_arr[ifile][icentr][ilayer])
            if N_pt<1: continue
            g_sigma_centr.append(root.TGraphErrors(N_pt, pt_arr[ifile][icentr][ilayer], Sigmas[ifile][icentr][ilayer],pt_arr_err[ifile][icentr][ilayer],Sigma_errs[ifile][icentr][ilayer]))
            g_sigma_centr[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{icentr}_l{ilayer}")
            Format_Graph(g_sigma_centr[-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_sigma_centr[-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            g_sigma_centr[-1].Draw('P')
            legends[-1].AddEntry(g_sigma_centr[-1],f"layer {ilayer}","p")

        legends[-1].Draw()
        g_sigma_f.append(g_sigma_centr)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{iOption1}.png")

In [14]:
print("const float sigma_"+xaxis[iOption0]+f"_pars[{N_files}][{N_centr}][{N_layers}][4]="+"\n{")
for fit_functions_ifile in fit_functions:
    print("    {")
    for fit_functions_icentr in fit_functions_ifile:
        print("        {")
        for fits in fit_functions_icentr:
            print("             {"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}, {round(fits.GetParameter(3),6)}"+"},")
        print("        },") 
    print("    },")      
print("};")        

const float sigma_phi_pars[1][5][4][4]=
{
    {
        {
             {0.005365, 0.036591, -3.94632, -0.00084},
             {0.005047, 0.035547, -3.917492, -0.000777},
             {0.004798, 0.038092, -4.085188, -0.000763},
             {0.0043, 0.035453, -4.164463, -0.000635},
        },
        {
             {0.00488, 0.02937, -3.487827, -0.000647},
             {0.004603, 0.028629, -3.497789, -0.000612},
             {0.004341, 0.030138, -3.66087, -0.000601},
             {0.003828, 0.02631, -3.620383, -0.000464},
        },
        {
             {0.004879, 0.0264, -3.431168, -0.000652},
             {0.004564, 0.025186, -3.413593, -0.000602},
             {0.004147, 0.025717, -3.477064, -0.000555},
             {0.003752, 0.022454, -3.523224, -0.000452},
        },
        {
             {0.006797, 0.022924, -4.251754, -0.001978},
             {0.006466, 0.020408, -4.099483, -0.001877},
             {0.006325, 0.027998, -5.216916, -0.00195},
             {0.005307, 0.027498, -

In [15]:
colors=[1,2,4,6,root.kGreen+2]
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.029)
            fit_functions1[-1].FixParameter(1,0.)
            fit_functions1[-1].FixParameter(2,0)
            #fit_functions1[-1].SetParLimits(2,-3.126,-0.023)
            fit_functions1[-1].SetLineColor(colors[ilayer%4])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [16]:
g_mean = []
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3",f"c3",1500,900)
    c3.Divide(3,2)
    g_mean_f = []
    min, max = -0.003, 0.003
    if iOption1==1: max, min = -1, 1
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        g_mean_centr = []
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_mean_layer = []
            N_pt=len(pt_arr[ifile][icentr][ilayer])
            g_mean_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][icentr][ilayer], Means[ifile][icentr][ilayer],pt_arr_err[ifile][icentr][ilayer],Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_mean_layer[-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
            g_mean_layer[-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","",0.6,5)
            g_mean_layer[-1].Draw('P')
            legends[-1].AddEntry(g_mean_layer[-1],f"layer {ilayer}","p")
            g_mean_centr.append(g_mean_layer)

        legends[-1].Draw()
        g_mean_f.append(g_mean_centr)
    g_mean.append(g_mean_f)
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{iOption1}.png")

In [17]:
print("const float mean_"+xaxis[iOption0]+f"_pars[{N_files}][{N_centr}][{N_layers}]="+"\n{")
for fit_functions_file in fit_mean_functions:
    print("    {")
    for fit_functions_syst in fit_functions_file:
        if sep_charge: print("        {"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)},\
    {round(fit_functions_syst[0+4].GetParameter(0),6)}, {round(fit_functions_syst[1+4].GetParameter(0),6)},{round(fit_functions_syst[2+4].GetParameter(0),6)}, {round(fit_functions_syst[3+4].GetParameter(0),6)},\
    {round(fit_functions_syst[0+8].GetParameter(0),6)}, {round(fit_functions_syst[1+8].GetParameter(0),6)},{round(fit_functions_syst[2+8].GetParameter(0),6)}, {round(fit_functions_syst[3+8].GetParameter(0),6)},\
    {round(fit_functions_syst[0+12].GetParameter(0),6)}, {round(fit_functions_syst[1+12].GetParameter(0),6)},{round(fit_functions_syst[2+12].GetParameter(0),6)}, {round(fit_functions_syst[3+12].GetParameter(0),6)}"+"},")
    print("    },")
print("};")

const float mean_phi_pars[1][5][4]=
{
    {
    },
};


In [18]:
outrootfile = root.TFile(f"output/delta_fits/deltasigma{iOption0}{sim}.root","RECREATE")
for g_sigma_f in g_sigma:
    for g_sigma_centr in g_sigma_f:
        for g_sigma_layer in g_sigma_centr:
            g_sigma_layer.Write()